#### CV of station features

In [2]:
import sys
sys.path.insert(0, '../../bosch_helper')
from bosch_helper import *

%matplotlib inline

/c3se/NOBACKUP/users/lyaa/conda_dir/miniconda/envs/kaggle/lib/python3.7/site-packages/sklearn/utils/__init__.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sequence


##### Load data up to benchmark 7

In [3]:
filename = pd.HDFStore('../hpopt_1/numeric_b1_b7_nf149.hdf')
print(filename.keys())
filename.close()

['/numeric', '/y_train']


In [4]:
x = pd.read_hdf('../hpopt_1/numeric_b1_b7_nf149.hdf', 'numeric')
y_train = pd.read_hdf('../hpopt_1/numeric_b1_b7_nf149.hdf', 'y_train')

##### Load station features

In [5]:
time_station = pd.read_hdf('time_station.hdf', 'time_station')

In [8]:
time_station.columns.tolist()

['time_start_S0',
 'time_start_S1',
 'time_start_S10',
 'time_start_S11',
 'time_start_S12',
 'time_start_S13',
 'time_start_S14',
 'time_start_S15',
 'time_start_S16',
 'time_start_S17',
 'time_start_S18',
 'time_start_S19',
 'time_start_S2',
 'time_start_S20',
 'time_start_S21',
 'time_start_S22',
 'time_start_S23',
 'time_start_S24',
 'time_start_S25',
 'time_start_S26',
 'time_start_S27',
 'time_start_S28',
 'time_start_S29',
 'time_start_S3',
 'time_start_S30',
 'time_start_S31',
 'time_start_S32',
 'time_start_S33',
 'time_start_S34',
 'time_start_S35',
 'time_start_S36',
 'time_start_S37',
 'time_start_S38',
 'time_start_S39',
 'time_start_S4',
 'time_start_S40',
 'time_start_S41',
 'time_start_S42',
 'time_start_S43',
 'time_start_S44',
 'time_start_S45',
 'time_start_S46',
 'time_start_S47',
 'time_start_S48',
 'time_start_S49',
 'time_start_S5',
 'time_start_S50',
 'time_start_S51',
 'time_start_S6',
 'time_start_S7',
 'time_start_S8',
 'time_start_S9',
 'station_start',
 'st

In [10]:
x = x.join(nhash)

In [12]:
x_train = x.loc['train']
x_test = x.loc['test']

In [ ]:
del x, nhash
gc.collect()

### CV score based on stratified KFold with repeated models

In [16]:
# CV
# specify parameters 
# 'booster':'gbtree'
params = {'max_depth':14, 'eta':0.03, 'silent':1, 'objective':'binary:logistic', 'nthread':20,
         'lambda':4, 'subsample':0.9, 'min_child_weight':5, 'booster':'gbtree', 'alpha':0,
         'base_score':0.0058, 'colsample_bytree':0.6}

# 'booster':'dart'
# params = {'max_depth':14, 'eta':0.03, 'silent':1, 'objective':'binary:logistic', 'nthread':20,
#         'lambda':4, 'subsample':0.9, 'min_child_weight':5, 'booster':'dart', 'alpha':0,
#         'base_score':0.0058, 'nthread':20, 'colsample_bytree':0.6, 
#         'sample_type':'uniform', 'normalize_type':'tree', 'rate_drop':0.1, 'skip_drop':0.2, 'one_drop':True}

cv_results, clfs, running_time = cross_val_predict_skf_rm_xgb(params, x_train, y_train, 
                                                              num_boost_round=80, n_splits=5, 
                                                              n_repeats=3, random_state=5870577, 
                                                              verbose_eval=True)

results = {'clfs': clfs, 'cv_results': cv_results, 'running_time': running_time}
save_pickle(results, 'results_benchmark_8_cv_5_hash.pickle')

KeyboardInterrupt: 

In [ ]:
cv_train_mean = cv_results['train'].mean(axis=1)
cv_train_std = cv_results['train'].std(axis=1)
cv_test_mean = cv_results['test'].mean(axis=1)
cv_test_std = cv_results['test'].std(axis=1)

In [ ]:
plt.figure(figsize=(14, 7))
plt.plot(np.arange(len(cv_train_mean)), cv_train_mean)
plt.fill_between(np.arange(len(cv_train_mean)), cv_train_mean-cv_train_std, cv_train_mean+cv_train_std, alpha=0.5)
plt.plot(np.arange(len(cv_train_mean)), cv_test_mean)
plt.fill_between(np.arange(len(cv_test_mean)), cv_test_mean-cv_test_std, cv_test_mean+cv_test_std, alpha=0.5)
plt.legend(['train', 'test'])

In [ ]:
# Train the model
dtrain = xgb.DMatrix(x_train, label=y_train)
params['seed'] = 28537894
clf = xgb.train(params, dtrain, num_boost_round=60,
               feval=mcc_eval, evals=[(dtrain, 'train')])

y_train_pred = clf.predict(dtrain)

# Find best threshold 
thresholds = np.linspace(0.01, 0.99, 400)
mcc = np.array([matthews_corrcoef(y_train, y_train_pred>thr) for thr in thresholds])
plt.plot(thresholds, mcc)
best_threshold = thresholds[mcc.argmax()]

print('Optimal MCC = {:.3f}'.format(mcc.max()))
print('Optimal threshold = {:.3f}'.format(best_threshold))

### Predict on test data

In [ ]:
dtest = xgb.DMatrix(x_test)
y_test_pred = clf.predict(dtest)
y_test_pred_int = (y_test_pred>best_threshold).astype(int)

sub = pd.read_csv("../../data/sample_submission.csv.zip", index_col=0)
sub["Response"] = y_test_pred_int
sub.to_csv("benchmark_8_submission_cv_5_hash.csv.gz", compression="gzip")

Only With MeanTimeDiff Features:
- LB: `n_estimators=60`: Private MCC = 0.44401, public MCC = 0.43569
- CV: `n_estimators=80`: Score mean = 0.448, std = 0.013
- CV: `n_estimators=60`: Score mean = 0.445491, std = 0.0133, MCC~[0.43218, 0.45880]

Time series features, excluding MeanTimeDiff:
- LB: `n_estimators=60`: Private MCC = 0.46212, public MCC = 0.44761
- CV: `n_estimators=80`: Score mean = 0.454509, std = 0.00586
- CV: `n_estimators=60`: Score mean = 0.4534794, std = 0.0067, MCC~[0.44682, 0.46014]

Time series features, including MeanTimeDiff:
- LB: `n_estimators=60`: Private MCC = 0.43169, public MCC = 0.42776
- CV: `n_estimators=80`: Score mean = 0.452, std = 0.009
- CV: `n_estimators=60`: Score mean = 0.449, std = 0.009, MCC~[0.440109, 0.458029]
- Apparently overfitting since CV is higher than LB

In [28]:
cv_test_mean[60]+cv_test_std[60]

cv_test_mean[60]-cv_test_std[60]

cv_test_mean[79]

cv_test_std[60]

0.008573770361371